![Ironhack logo](https://i.imgur.com/1QgrNNw.png)

# Lab | Parallelization

## Introduction

In the parallelization lesson, we saw two examples of how we could implement parallelization using the multiprocessing library to speed up the tasks that we need to perform.

This lab will combine parallelization with some of the other topics you have learned in the Intermediate Python module of this program (list comprehensions, requests library, functional programming, web scraping, etc.). You will write code that extracts a list of links from a web page, requests each URL, and then indexes the page referenced by each link - both sequentially and in parallel.

## Resources

- [Multiprocessing Library Documentation](https://docs.python.org/3/library/multiprocessing.html?highlight=multiprocessing#module-multiprocessing)
- [Python Parallel Computing (in 60 Seconds or less)](https://dbader.org/blog/python-parallel-computing-in-60-seconds)
- [Python Multiprocessing: Pool vs Process – Comparative Analysis](https://www.ellicium.com/python-multiprocessing-pool-process/)

# Parallelization Lab

In this lab, you will be leveraging several concepts you have learned to obtain a list of links from a web page and crawl and index the pages referenced by those links - both sequentially and in parallel. Follow the steps below to complete the lab.

## Step 1: Use the requests library to retrieve the content from the URL below.

In [1]:
import requests
url = 'https://en.wikipedia.org/wiki/Data_science'

In [2]:
html = requests.get(url).content;

## Step 2: Use BeautifulSoup to extract a list of all the unique links on the page.

In [3]:
from bs4 import BeautifulSoup;

In [4]:
soup = BeautifulSoup(html, "lxml");
link_tags = soup.find_all('a', href=True);
links = list(set([link['href'] for link in link_tags]));

## Step 3: Use list comprehensions with conditions to clean the link list.

There are two types of links, absolute and relative. Absolute links have the full URL and begin with http while relative links begin with a forward slash (/) and point to an internal page within the wikipedia.org domain. Clean the respective types of URLs as follows.

- Absolute Links: Create a list of these and remove any that contain a percentage sign (%).
- Relative Links: Create a list of these, add the domain to the link so that you have the full URL, and remove any that contain a percentage sign (%).
- Combine the list of absolute and relative links and ensure there are no duplicates.

In [5]:
domain = 'http://wikipedia.org';

In [6]:
absolute = [link for link in links if link.startswith('http') and '%' not in link];

In [7]:
relative = [domain + link for link in links if link.startswith('/') and not link.startswith('//') and '%' not in link];

In [8]:
urls = list(set(absolute + relative));

## Step 4: Use the os library to create a folder called *wikipedia* and make that the current working directory.

In [9]:
import os;

In [10]:
folder = './wikipedia'

if not os.path.exists(folder):
    os.makedirs(folder)

os.chdir(folder)
path = os.getcwd() 

## Step 5: Write a function called index_page that accepts a link and does the following.

- Tries to request the content of the page referenced by that link.
- Slugifies the filename using the `slugify` function from the [python-slugify](https://pypi.org/project/python-slugify/) library and adds a .html file extension.
    - If you don't already have the python-slugify library installed, you can pip install it as follows: `$ pip install python-slugify`.
    - To import the slugify function, you would do the following: `from slugify import slugify`.
    - You can then slugify a link as follows `slugify(link)`.
- Creates a file in the wikipedia folder using the slugified filename and writes the contents of the page to the file.
- If an exception occurs during the process above, just `pass`.

In [11]:
from slugify import slugify

In [12]:
#if you have window you will need to import all libraries and define all variables used inside the function
def index_page(url):
    #import requests
    #from slugify import slugify
    #import os
    #path = os.getcwd()
    try:
        html = requests.get(url).content
        filename = slugify(url).lower() + ".html"
        with open(path + '/' + filename, "wb") as f:
            f.write(html)
    except:
        pass

## Step 6: Sequentially loop through the list of links, running the index_page function each time.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [13]:
%%time
for url in urls:
    index_page(url)

Wall time: 7min 50s


## Step 7: Perform the page indexing in parallel and note the difference in performance.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [14]:
import multiprocessing
# If you are using windows use the multiprocess library 

In [15]:
%%time
pool = multiprocessing.Pool()
#pool = multiprocess.Pool()
result = pool.map(index_page, urls)
pool.terminate()

Wall time: 1min 10s


**BONUS**: Create a function that counts how many files are there in the wikipedia folder using the `os` module. 

Delete the files from the folder before you run and perform the above solution asynchronously. 

Use your function to check how many files are being downloaded.

In [16]:
def count_itens(path):
    return len(os.listdir(path))

In [17]:
pool = multiprocessing.Pool()
#pool = multiprocess.Pool()
result = pool.map_async(index_page, urls)

In [20]:
count_itens(path)

24

In [21]:
count_itens(path)

30